In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import fastai
fastai.__version__

'1.0.28'

In [3]:
from fastai import *
from fastai_audio import *
from fastai.vision import create_cnn, models

In [4]:
DATA = Path('data')
AUDIO = DATA/'freesound/audio_44KHz'
CSV = DATA/'freesound/audio_44KHz/train.csv'

In [5]:
df = pd.read_csv(CSV)
df['fname'] = df['fname'].str[:-4]
df.head(3)

,fname,label,manually_verified
0,00044347,Hi-hat,0
1,001ca53d,Saxophone,1
2,002d256b,Trumpet,0


In [6]:
n_fft = 2048
n_hop = 512
n_mels = 96
sr = 44100
top_db = 60.0

bs = 16

In [7]:
mel_spec_tfm = Compose([
    Spectrogram(n_fft=n_fft, n_hop=n_hop),
    FrequencyToMel(n_mels=n_mels, n_fft=n_fft, sr=sr, f_min=0., f_max=None),
    SpectrumToDb(ref='max', top_db=top_db, normalized=True)
])

def inputs_tfm(inputs):
    x, y = inputs
    x = mel_spec_tfm(x)
    # repeat over 3 'channels' to match expected resnet input shape
    x = x.unsqueeze_(1).repeat(1, 3, 1, 1)
    return x, y

tfms = [inputs_tfm]

In [8]:
data = (AudioItemList
            .from_df(df, path=AUDIO, folder='train', suffix='.wav')
            .random_split_by_pct()
            .label_from_df()
            .databunch(bs=bs, tfms=tfms))

In [9]:
xs, ys = data.train_dl.one_batch()
xs.shape, ys.shape, xs.min(), xs.max()

(torch.Size([16, 3, 96, 2584]), torch.Size([16]), tensor(-0.4356), tensor(1.))

In [10]:
model_name = 'freesound_44khz_2048_512_96_60db_v1'

learn = create_cnn(data, models.resnet18, metrics=[error_rate, mapk])

In [ ]:
learn.fit_one_cycle(8)
learn.save(model_name + '-stage-1')